## $\text{Introduction}$

---



$\text{Name : Kartik Srinivas}$


$\text{Roll : ES20BTECH11015}$

$\text{Welcome to the analysis of the Robust LP Problem !}$

1.   $\text{All the equations have been coded in LaTeX by me}$ 
2.  $\text{The necessary explanations have been provided }$ 


In [154]:
import numpy as np
import cvxpy as cp

### $\text{(a) Convexity}$

---

\begin{align}
f(x) = \underset{c \in C}{\sup}  \ c^T x = \underset{c \in C}{\sup} \ \ \text{(Affine)}
\\
A x \ge b
\end{align}

This function is a maximum of affine functions = maximum of convex functions  = convex



Constraints are polyhedra, so they are convex

In [155]:
np.random.seed(10)
(m,n) = (30,10)
A = np.random.rand(m,n)
A = np.asmatrix(A)
b = np.random.rand(m,1)
b = np.asmatrix(b)
c_nom = np.ones((n,1)) + np.random.rand(n,1)
c_nom = np.asmatrix(c_nom)

In [166]:
print(c_nom.shape)
print(A.shape)
print(b.shape)
print(type(A))

(10, 1)
(30, 10)
(30, 1)
<class 'numpy.matrix'>


In [157]:
x = cp.Variable((10,1))


### $\text{Framing F and g}$

---

F needs to be formed according to the constraints defined by 
\begin{align}
  c \le 1.25  \ c_{nom}
  \\
  -c \le -0.75 \  c_{nom}
  \\
  \vec{1}^T c \le \vec{1}^T c_{nom}
  \\
  \vec{-1}^T c \le \vec{-1}^T c_{nom}
  \\
\end{align}
In order to make this in matrix form we do the following
\begin{align}
    \begin{bmatrix}
        I_{n \times n} \\
        -I_{n \times n} \\
        \vec{1}^T c \\
        \vec{-1}^T c
    \end{bmatrix}
    \cdot
    \vec{c}
     = 
     \begin{bmatrix}
     1.25 \ c_{nom} \\
     -0.75 \ c_{nom} \\
     \vec{1}^T c_{nom} \\
     \vec{-1}^T c_{nom}
     \end{bmatrix}
\end{align}

In [158]:
#Now to generate the values of F and g
I = np.eye(n)
Stacked_identitites = np.block([
    [I],
    [-I]
])
F = np.vstack((np.vstack((Stacked_identitites, np.ones(n))), -1 *np.ones(n)))
print(F)
print(F.shape)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [-1. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -1. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -1. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -1. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -1. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -1. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -1. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -1. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -1. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]]
(22, 10)


In [160]:
#to calculate g
#print(c_nom)
stacked_noms = np.vstack( ( (1.25 * c_nom), (-0.75 * c_nom) )  )
# print(stacked_noms.shape)
sum = np.sum(c_nom)
#print(sum)
g_1 = np.vstack((stacked_noms, 1.1*sum))
g = np.vstack((g_1, -0.9*sum))
print(g)
print(g.shape)


[[  2.00180047]
 [  1.37159062]
 [  1.6109186 ]
 [  2.15099988]
 [  1.93850758]
 [  2.29822127]
 [  1.97541419]
 [  1.48071467]
 [  2.01937765]
 [  2.35869386]
 [ -1.20108028]
 [ -0.82295437]
 [ -0.96655116]
 [ -1.29059993]
 [ -1.16310455]
 [ -1.37893276]
 [ -1.18524851]
 [ -0.8884288 ]
 [ -1.21162659]
 [ -1.41521632]
 [ 16.90149013]
 [-13.82849192]]
(22, 1)


In [161]:
lam = cp.Variable(g.shape)

### $\text{(b),(c) Framing a Dual}$

---

THe dual of the problem is 
\begin{align}
 f(x) = \max g(\lambda)
 \\
 \lambda \ge 0 
\end{align}
Where :-
\begin{align}
  g(\lambda) = \underset{c}{min}  \ [c^T x + \lambda^T(Fc  - g)]
\end{align}
Unbounded unless coefficient of c is zero !

\begin{align}
F^T \lambda = x 
\\
\therefore g(\lambda) = -\lambda^T g
\end{align}

Therefore the solution of the primal 
\begin{align}
 \underset{x}{\min} \ (\underset{\lambda}{\max} \ g(\lambda)) =  \underset{x, \lambda}{\min} \ ( \lambda^T g)
 \\
 \lambda \ge 0 
 \\
 F^T \lambda = x 
 \\
 Ax \ge b
\end{align}




In [167]:
objective = cp.Minimize(lam.T @ g)
constraints = [F.T @ lam == x, lam >= 0 , A@x >= b]
prob = cp.Problem(objective, constraints)
result_dual_form = prob.solve()
print(result_dual_form)
x_robust = x.value

3.1659610520520705


In [169]:
#solving the primal using c_nom
x2 = cp.Variable((10,1))
test_obj = cp.Minimize(c_nom.T @ x2)
test_constraints = [A@x2 >= b]
test_prob = cp.Problem(test_obj,test_constraints)
result_nominal = test_prob.solve()
print("Result of Nominal C = ",result_nominal)

Result of Nominal C =  2.1092714620826003


In [165]:
print("Result of Robust C = ", (c_nom.T @ x_robust)[0,0])

Result of Robust C =  2.5232088648898556


### $\text{Final Values}$ 

---

\begin{align}
 c_{nom}^T x_{robust} = 2.52
 \\
 c_{nom}^T x_{nominal} = 2.109
 \\
 f(x)_{min} = 3.165 
\end{align}